In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('train.csv')

In [3]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = train_data['label']

In [5]:
labels.values.reshape((42000,1))

array([[1],
       [0],
       [1],
       ..., 
       [7],
       [6],
       [9]], dtype=int64)

In [6]:
#Need to change this to a hot ecnoded label
labels_he = np.zeros((labels.size, labels.max()+1))
labels_he[np.arange(labels.size),labels] = 1
labels_he.shape

(42000, 10)

In [7]:
#transform to [28,28 for each row]
train_data = train_data.drop('label',axis=1)
train_data = train_data.as_matrix()
train_data

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [8]:
train_data.shape

(42000, 784)

In [9]:
#from sklearn.preprocessing import MinMaxScaler

In [10]:
#scaler = MinMaxScaler()

In [11]:
#pixel_data = train.drop('label',axis=1)
#pixel_data = scaler.fit_transform(pixel_data)

In [12]:
#HELPER

#Init Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [13]:
#Init Bias
def init_bias(shape):
    init_bias_vals = tf.constant(.1,shape=shape)
    return tf.Variable(init_bias_vals)

In [14]:
#2D Convolution
def conv2d(x,W):
    # x --> input tensor, [batch,H,W,Channels]
    # W --> [filter H, filter W, Channels In, Channels Out]
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [15]:
#Pooling
def max_pool_2by2(x):
    # x --> [batch,h,w,c]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [16]:
#Convultional Layer
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [17]:
#Normal (Fully Connected)
def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [18]:
#Placeholders
x = tf.placeholder(tf.float32,shape=[None,784])

In [19]:
y_true = tf.placeholder(tf.float32,shape=[None,10])

In [20]:
#Layers
x_image = tf.reshape(x,[-1,28,28,1])

In [21]:
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [22]:
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [23]:
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [24]:
#Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

In [25]:
y_pred = normal_full_layer(full_one_dropout,10)

In [26]:
#Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [27]:
#OptimizerAdam
optimizer = tf.train.AdamOptimizer(learning_rate=.001)
train = optimizer.minimize(cross_entropy)

In [28]:
init = tf.global_variables_initializer()

In [29]:
saver = tf.train.Saver()

In [30]:
def get_next_batch(step=0,size=32,x_data=None,labels=None):
    return x_data[step*size:step*size+size], labels[step*size:step*size+size]

In [ ]:
with tf.Session() as sess:
    
    sess.run(init)

    for step in range(420):
        #TO DO - get this to run in batches, not as a whole training set
        #make into batches
        print(step)
        batch_x, batch_y = get_next_batch(step,100,train_data,labels_he)
        print(batch_x.shape)
        print(batch_y.shape)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})
        
        if step==419:
            print("On Step {}".format(step))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:train_data,y_true:labels_he,hold_prob:1.0}))
    
    saver.save(sess,'./MINSTmodel.ckpt')

0
(100, 784)
(100, 10)
1
(100, 784)
(100, 10)
2
(100, 784)
(100, 10)
3
(100, 784)
(100, 10)
4
(100, 784)
(100, 10)
5
(100, 784)
(100, 10)
6
(100, 784)
(100, 10)
7
(100, 784)
(100, 10)
8
(100, 784)
(100, 10)
9
(100, 784)
(100, 10)
10
(100, 784)
(100, 10)
11
(100, 784)
(100, 10)
12
(100, 784)
(100, 10)
13
(100, 784)
(100, 10)
14
(100, 784)
(100, 10)
15
(100, 784)
(100, 10)
16
(100, 784)
(100, 10)
17
(100, 784)
(100, 10)
18
(100, 784)
(100, 10)
19
(100, 784)
(100, 10)
20
(100, 784)
(100, 10)
21
(100, 784)
(100, 10)
22
(100, 784)
(100, 10)
23
(100, 784)
(100, 10)
24
(100, 784)
(100, 10)
25
(100, 784)
(100, 10)
26
(100, 784)
(100, 10)
27
(100, 784)
(100, 10)
28
(100, 784)
(100, 10)
29
(100, 784)
(100, 10)
30
(100, 784)
(100, 10)
31
(100, 784)
(100, 10)
32
(100, 784)
(100, 10)
33
(100, 784)
(100, 10)
34
(100, 784)
(100, 10)
35
(100, 784)
(100, 10)
36
(100, 784)
(100, 10)
37
(100, 784)
(100, 10)
38
(100, 784)
(100, 10)
39
(100, 784)
(100, 10)
40
(100, 784)
(100, 10)
41
(100, 784)
(100, 10)
42

333
(100, 784)
(100, 10)
334
(100, 784)
(100, 10)
335
(100, 784)
(100, 10)
336
(100, 784)
(100, 10)
337
(100, 784)
(100, 10)
338
(100, 784)
(100, 10)
339
(100, 784)
(100, 10)
340
(100, 784)
(100, 10)
341
(100, 784)
(100, 10)
342
(100, 784)
(100, 10)
343
(100, 784)
(100, 10)
344
(100, 784)
(100, 10)
345
(100, 784)
(100, 10)
346
(100, 784)
(100, 10)
347
(100, 784)
(100, 10)
348
(100, 784)
(100, 10)
349
(100, 784)
(100, 10)
350
(100, 784)
(100, 10)
351
(100, 784)
(100, 10)
352
(100, 784)
(100, 10)
353
(100, 784)
(100, 10)
354
(100, 784)
(100, 10)
355
(100, 784)
(100, 10)
356
(100, 784)
(100, 10)
357
(100, 784)
(100, 10)
358
(100, 784)
(100, 10)
359
(100, 784)
(100, 10)
360
(100, 784)
(100, 10)
361
(100, 784)
(100, 10)
362
(100, 784)
(100, 10)
363
(100, 784)
(100, 10)
364
(100, 784)
(100, 10)
365
(100, 784)
(100, 10)
366
(100, 784)
(100, 10)
367
(100, 784)
(100, 10)
368
(100, 784)
(100, 10)
369
(100, 784)
(100, 10)
370
(100, 784)
(100, 10)
371
(100, 784)
(100, 10)
372
(100, 784)
(100, 10)


In [ ]:
#lets use the model now to predict
test = pd.read_csv('test.csv')

In [ ]:
test_normalized = scaler.fit_transform(test)

In [ ]:
predictions = tf.nn.softmax(tf.matmul(x,W) + b)

In [ ]:
with tf.Session() as sess:
    
    saver.restore(sess,'MINSTmodel.ckpt')

    prediction=tf.argmax(y,1)
    pred = prediction.eval(feed_dict={x: test_normalized})

In [ ]:
test = pd.DataFrame()

In [ ]:
test['Label'] = pred

In [ ]:
test['Label'].size

In [ ]:
test['ImageId'] = np.arange(1,test['Label'].size+1)

In [ ]:
test.head()

In [ ]:
test.to_csv('eval.csv')